In [40]:
from typing import Tuple, List, Text, Dict, Any, Iterator
import keras
from keras.models import Model
from keras.layers import Input
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.merge import Concatenate
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.core import Activation, Dropout, Flatten, Dense
from keras.layers.normalization import BatchNormalization
import numpy as np
from keras.optimizers import Adam
import tensorflow as tf
from keras import backend as K
import os
from tqdm import tqdm

In [41]:
def generator(in_shape: Tuple[int,int,int], out_shape: Tuple[int,int,int], filters: int):
    
    input_tensor = Input(shape = in_shape)
    noise = Input(shape = (in_shape[0], in_shape[1], 1))
    output_ch = out_shape[2]
        
    #Encoder
    x = BatchNormalization()(Conv2D(filters*1, kernel_size = (5, 5), strides = (2, 2), padding = "same")(Concatenate()([input_tensor, noise])))
    x = LeakyReLU(0.2)(x); e1 = x
    x = BatchNormalization()(Conv2D(filters*2, kernel_size = (5, 5), strides = (2, 2), padding = "same")(x))
    x = LeakyReLU(0.2)(x); e2 = x
    x = BatchNormalization()(Conv2D(filters*4, kernel_size = (5, 5), strides = (2, 2), padding = "same")(x))
    x = LeakyReLU(0.2)(x); e3 = x
    x = BatchNormalization()(Conv2D(filters*8, kernel_size = (5, 5), strides = (2, 2), padding = "same")(x))
    x = LeakyReLU(0.2)(x); e4 = x
    x = BatchNormalization()(Conv2D(filters*8, kernel_size = (4, 4), strides = (2, 2), padding = "same")(x))
    x = LeakyReLU(0.2)(x); e5 = x
    x = BatchNormalization()(Conv2D(filters*8, kernel_size = (4, 4), strides = (2, 2), padding = "same")(x))
    x = LeakyReLU(0.2)(x); e6 = x
    x = BatchNormalization()(Conv2D(filters*8, kernel_size = (4, 4), strides = (2, 2), padding = "same")(x))
    x = LeakyReLU(0.2)(x); e7 = x
    x = BatchNormalization()(Conv2D(filters*8, kernel_size = (4, 4), strides = (2, 2), padding = "same")(x))
    x = LeakyReLU(0.2)(x); e8 = x
    
    #Decoder
    x = BatchNormalization()(Conv2DTranspose(filters*8, kernel_size = (4, 4), strides = (2, 2), padding = "same")(x))
    x = LeakyReLU(0.2)(x); x = Concatenate()([Dropout(0.5)(x), e7])
    x = BatchNormalization()(Conv2DTranspose(filters*8, kernel_size = (4, 4), strides = (2, 2), padding = "same")(x))
    x = LeakyReLU(0.2)(x); x = Concatenate()([Dropout(0.5)(x), e6])
    x = BatchNormalization()(Conv2DTranspose(filters*8, kernel_size = (4, 4), strides = (2, 2), padding = "same")(x))
    x = LeakyReLU(0.2)(x); x = Concatenate()([Dropout(0.5)(x), e5])
    x = BatchNormalization()(Conv2DTranspose(filters*8, kernel_size = (4, 4), strides = (2, 2), padding = "same")(x))
    x = LeakyReLU(0.2)(x); x = Concatenate()([Dropout(0.5)(x), e4])
    x = BatchNormalization()(Conv2DTranspose(filters*4, kernel_size = (4, 4), strides = (2, 2), padding = "same")(x))
    x = LeakyReLU(0.2)(x); x = Concatenate()([Dropout(0.5)(x), e3])
    x = BatchNormalization()(Conv2DTranspose(filters*2, kernel_size = (4, 4), strides = (2, 2), padding = "same")(x))
    x = LeakyReLU(0.2)(x); x = Concatenate()([Dropout(0.5)(x), e2])
    x = BatchNormalization()(Conv2DTranspose(filters*1, kernel_size = (4, 4), strides = (2, 2), padding = "same")(x))
    x = LeakyReLU(0.2)(x); x = Concatenate()([Dropout(0.5)(x), e1])
    x = Conv2DTranspose(output_ch, kernel_size = (4, 4), strides = (2, 2), padding = "same")(x)
    x = Activation("tanh")(x)
    
    unet = Model(inputs=[input_tensor, noise], outputs = [x])
    return unet

In [27]:
net = generator((512, 512, 10), (512,512,3), 64)
net.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_27 (InputLayer)           (None, 512, 512, 10) 0                                            
__________________________________________________________________________________________________
input_28 (InputLayer)           (None, 512, 512, 1)  0                                            
__________________________________________________________________________________________________
concatenate_38 (Concatenate)    (None, 512, 512, 11) 0           input_27[0][0]                   
                                                                 input_28[0][0]                   
__________________________________________________________________________________________________
conv2d_61 (Conv2D)              (None, 256, 256, 64) 17664       concatenate_38[0][0]             
__________

In [28]:
def discriminator(in_shape: Tuple[int,int,int], y_shape: Tuple[int,int,int], filters: int):
    input_tensor = Input(shape=in_shape)
    input_y = Input(shape=y_shape)
    x = LeakyReLU(0.2)(Conv2D(filters*1, kernel_size=(4, 4), strides=(2, 2), padding="same")(Concatenate()([input_tensor, input_y])))
    x = LeakyReLU(0.2)(BatchNormalization()(Conv2D(filters*2, kernel_size=(4, 4), strides=(2, 2), padding="same")(x)))
    x = LeakyReLU(0.2)(BatchNormalization()(Conv2D(filters*4, kernel_size=(4, 4), strides=(2, 2), padding="same")(x)))
    x = LeakyReLU(0.2)(BatchNormalization()(Conv2D(filters*8, kernel_size=(4, 4), strides=(2, 2), padding="same")(x)))
    x = LeakyReLU(0.2)(BatchNormalization()(Conv2D(filters*8, kernel_size=(4, 4), strides=(1, 1), padding="same")(x)))
    x = LeakyReLU(0.2)(BatchNormalization()(Conv2D(filters*4, kernel_size=(4, 4), strides=(1, 1), padding="same")(x)))
    x = Activation("sigmoid")(Conv2D(1, kernel_size=(4, 4), strides=(1, 1), padding="same")(x))

    disc = Model(inputs=[input_tensor, input_y], outputs=[x])
    return disc


In [29]:
disc = discriminator((512, 512, 10), (512, 512, 3), 64)
disc.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_29 (InputLayer)           (None, 512, 512, 10) 0                                            
__________________________________________________________________________________________________
input_30 (InputLayer)           (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
concatenate_46 (Concatenate)    (None, 512, 512, 13) 0           input_29[0][0]                   
                                                                 input_30[0][0]                   
__________________________________________________________________________________________________
conv2d_69 (Conv2D)              (None, 256, 256, 64) 13376       concatenate_46[0][0]             
__________

In [30]:
def cGAN_Model(gen, disc):
    input_gen = Input((512, 512, 10))
    noise = Input((512, 512, 1))
    output_gen = gen([input_gen, noise])
    #input_disc = Concatenate()([input_gen, output_gen])
    disc.trainable = False
    output_disc = disc([input_gen, output_gen])
    cGAN = Model(inputs = [input_gen, noise], outputs = [output_gen, output_disc])
    return cGAN

In [31]:
cgan = cGAN_Model(net, disc)
cgan.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_31 (InputLayer)           (None, 512, 512, 10) 0                                            
__________________________________________________________________________________________________
input_32 (InputLayer)           (None, 512, 512, 1)  0                                            
__________________________________________________________________________________________________
model_13 (Model)                (None, 512, 512, 3)  55994435    input_31[0][0]                   
                                                                 input_32[0][0]                   
__________________________________________________________________________________________________
model_14 (Model)                (None, 32, 32, 1)    9069761     input_31[0][0]                   
          

In [32]:
def generate_noise(in_shape: Tuple[int,int,int]):
    return np.random.normal(0, 1, size=in_shape)

In [96]:
print(generate_noise((5, 5, 1)))

[[[-2.00298566]
  [ 0.92127517]
  [ 0.1392467 ]
  [ 2.10072708]
  [-0.22242183]]

 [[-0.86809901]
  [-2.06512589]
  [ 1.17634411]
  [-0.78937999]
  [-0.33870604]]

 [[-0.37542468]
  [-0.3722606 ]
  [-2.96793816]
  [-0.19585321]
  [ 0.33705055]]

 [[ 1.46118661]
  [-0.59830543]
  [ 0.35474009]
  [ 1.68559424]
  [-0.28919315]]

 [[ 1.3459891 ]
  [-0.01493187]
  [ 0.83713128]
  [ 1.05288414]
  [-0.24854697]]]


In [33]:
def l1_loss(y_true, y_pred): #MAE
    return  tf.reduce_mean(tf.abs(y_true - y_pred))

def discriminator_on_generator_loss(y_true, y_pred): #Binary Cross Entropy
    BATCH_SIZE=10
    
    #K.mean(K.binary_crossentropy(K.flatten(y_pred), K.flatten(y_true)), axis = -1)
    
    return K.mean(K.binary_crossentropy(K.flatten(y_pred), K.ones_like(K.flatten(y_pred))), axis=-1)



cgan_optimizer = Adam(lr=1E-3, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
gen_optimizer = keras.optimizers.Adam(lr=0.0002, beta_1=0.5, beta_2=0.999) 
loss_weights = [10, 1]
disc_optimizer = Adam(lr=1E-3, beta_1=0.9, beta_2=0.999, epsilon=1e-08)


In [34]:
gen_model = generator((512, 512, 10), (512, 512, 3), 64)

print("Generator Summary")
gen_model.summary()

print("Discriminator Summary")
disc_model = discriminator((512, 512, 10), (512, 512, 3), 64)
disc_model.summary()

print("Complete Model Summary")
cgan_model = cGAN_Model(gen_model, disc_model)
cgan_model.summary()

gen_model.compile(loss = 'mae', optimizer = gen_optimizer)
disc_model.compile(loss = 'binary_crossentropy', optimizer = disc_optimizer)
cgan_model.compile(loss =[l1_loss, discriminator_on_generator_loss], loss_weights = loss_weights, optimizer = cgan_optimizer)

print(cgan_model.metrics)

Generator Summary
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_33 (InputLayer)           (None, 512, 512, 10) 0                                            
__________________________________________________________________________________________________
input_34 (InputLayer)           (None, 512, 512, 1)  0                                            
__________________________________________________________________________________________________
concatenate_47 (Concatenate)    (None, 512, 512, 11) 0           input_33[0][0]                   
                                                                 input_34[0][0]                   
__________________________________________________________________________________________________
conv2d_76 (Conv2D)              (None, 256, 256, 64) 17664       concatenate_47[0][0]      

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_35 (InputLayer)           (None, 512, 512, 10) 0                                            
__________________________________________________________________________________________________
input_36 (InputLayer)           (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
concatenate_55 (Concatenate)    (None, 512, 512, 13) 0           input_35[0][0]                   
                                                                 input_36[0][0]                   
__________________________________________________________________________________________________
conv2d_84 (Conv2D)              (None, 256, 256, 64) 13376       concatenate_55[0][0]             
__________

In [35]:
#Data Preprocessing:
X_train_B = np.load('D:/dev/GeoGAN/DataSet/Data/Resized/npy/Shuffled/X_01Norm.npy')
X_train_A = np.load('D:/dev/GeoGAN/DataSet/Data/Resized/npy/Shuffled/Y_01Norm.npy')

In [45]:
#Train function : 
# 1. Generate images 
# 2. Train Discriminator as per Conditional GANs 
# 3. discriminator.trainable = False
# 4. Train DCGAN model 

d_loss = []
dc_gan_loss = []
dc = 0 
d_l = 0

def train(epochs, batch_size):
    Y_fake = np.zeros((batch_size,32,32,1))
    Y_real = np.random.uniform(0.7, 1, (batch_size,32,32,1))
    
    for num_epochs in range(epochs):
        for num_batch in tqdm(range(int(X_train_B.shape[0]/batch_size))):
            
            X_before = X_train_B[num_batch*batch_size : (num_batch + 1 )*batch_size , : , : , :]            
            X_after = X_train_A[num_batch*batch_size : (num_batch + 1 )*batch_size , : , : , : ]
            
            noise = generate_noise((1, 512, 512, 1))
            gen_images = gen_model.predict([X_before, noise])
        
            disc_model.trainable = True
            
            #d_loss_real = disc_model.train_on_batch(np.concatenate([X_before, X_after], axis = -1), Y_real)            
            #d_loss_fake = disc_model.train_on_batch(np.concatenate([X_before, gen_images], axis = -1), Y_fake)
            d_loss_real = disc_model.train_on_batch([X_before, X_after], Y_real)
            d_loss_fake = disc_model.train_on_batch([X_before, gen_images], Y_fake)
            
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
       
            disc_model.trainable = False
            for _ in range(2):
                dc_gan_loss = cgan_model.train_on_batch([X_before, noise], [X_after, Y_real])


In [46]:
train(2, 1)


  0%|                                                                                          | 0/291 [00:00<?, ?it/s]D:\Anaconda3\lib\site-packages\keras\engine\training.py:479: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'

 22%|█████████████████▊                                                               | 64/291 [02:21<07:10,  1.90s/it]


 45%|████████████████████████████████████▎                                           | 132/291 [04:30<04:58,  1.88s/it]


 69%|███████████████████████████████████████████████████████▎                        | 201/291 [06:41<02:50,  1.90s/it]

KeyboardInterrupt: 

In [23]:
#Save weights of generator and discriminator 
path_gen = os.path.join('weights','60_epochs_gen.h5')
path_disc = os.path.join('weights','60_epochs_disc.h5')
gen_model.save_weights(path_gen)
disc_model.save_weights(path_disc)

NameError: name 'os' is not defined

In [ ]:
#Load weights of generator and discriminator 
path_gen = os.path.join('train_weights_p2p','150_epochs_gen.h5py')
#path_disc = os.path.join('train_weights_p2p','150_epochs_disc.h5')
gen_model.load_weights(path_gen)
#disc_model.load_weights(path_disc)